## **농구선수  포지션  예측하기**
- 선수 시합 기록 데이터를 학습하여 선수 포지션 예측하는 모델 만들기

## **[방법1-1] 데이터셋 준비하기**

- pandas를 통해서 잘라서 씀

### 0. 데이터 다운로드 하기

In [1]:
# 구글 Colab에서 공유된 파일 다운로드해서 가져오기 - basketball.csv파일
!gdown 17O1XzwW9wqNFeidLzCRj6SeWaLLiGPY9

Downloading...
From: https://drive.google.com/uc?id=17O1XzwW9wqNFeidLzCRj6SeWaLLiGPY9
To: /content/basketball.csv
100% 2.89k/2.89k [00:00<00:00, 5.59MB/s]


### 1. 라이브러리 및 데이터 로드하기

In [2]:
# 필요한 패키지 가져오기
import pandas as pd

In [3]:
# 데이터 가져오기 - "basketball.csv" 파일을 read_csv()를 사용하여 읽어 df에 저장한다.

df = pd.read_csv("basketball.csv")              # ... 코드 작성
df.head()              # ... 코드 작성

,Player,Pos,3P,TRB,BLK
0,Denzel Valentine,SG,1.3,2.6,0.1
1,Kyle Korver,SG,2.4,2.8,0.3
2,Troy Daniels,SG,2.1,1.5,0.1
3,Tim Hardaway,SG,1.9,2.8,0.2
4,Dewayne Dedmon,C,0.0,6.5,0.8


In [4]:
# 예측을 위한 타겟항목 'Pos' 컬럼의 항목 별 갯수 파악하기
df["Pos"].value_counts()               # ... 코드 작성

Pos
SG    50
C     50
Name: count, dtype: int64

### 2. 특성/타켓 항목 정하고 학습/테스트 데이터셋 준비하기

In [5]:
# df 에서 훈련데이터 지정하기 : 80 개 ( iloc 사용)
train = df.iloc[:80, :]             # ... 코드 작성
train

,Player,Pos,3P,TRB,BLK
0,Denzel Valentine,SG,1.3,2.6,0.1
1,Kyle Korver,SG,2.4,2.8,0.3
2,Troy Daniels,SG,2.1,1.5,0.1
3,Tim Hardaway,SG,1.9,2.8,0.2
4,Dewayne Dedmon,C,0.0,6.5,0.8
...,...,...,...,...,...
75,Victor Oladipo,SG,1.9,4.3,0.3
76,Willie Cauley-Stein,C,0.0,4.5,0.6
77,Brook Lopez,C,1.8,5.4,1.7
78,Josh Richardson,SG,1.4,3.2,0.7


In [6]:
# train 데이터 형태파악하기
train.shape

(80, 5)

In [7]:
# df_bball 에서 테스트데이터 지정하기 : 80개 이외 남은데이터
test = df.iloc[80:, :]               # ... 코드 작성
test

,Player,Pos,3P,TRB,BLK
80,JaVale McGee,C,0.0,3.2,0.9
81,Manu Ginobili,SG,1.3,2.3,0.2
82,Nene Hilario,C,0.0,4.2,0.6
83,Evan Fournier,SG,1.9,3.1,0.1
84,Georgios Papagiannis,C,0.0,3.9,0.8
85,Anthony Davis,C,0.5,11.8,2.2
86,Tarik Black,C,0.0,5.1,0.7
87,Jamal Crawford,SG,1.4,1.6,0.2
88,Jordan Clarkson,SG,1.4,3.0,0.1
89,Marcin Gortat,C,0.0,10.4,0.7


In [8]:
# test 데이터 형태파악하기
test.shape

(20, 5)

In [9]:
# select data features to be used in train - 학습을 위해 X_train 특성항목 지정하기 (데이터프레임- 2차원)
X_train = train[['3P', 'BLK', 'TRB']]
# select target - 학습을 위해 y_train  타겟항목 지정하기 (시리즈로-1차원)
y_train = train['Pos']

# 예측을 위해 X_test 특성항목 지정하기 (데이터프레임- 2차원)
X_test = test[['3P', 'BLK', 'TRB']]           #... 코드 작성

# y_test 타켓항목 지정하기  (시리즈로-1차원)
y_test = test['Pos']                #... 코드 작성

## **[방법1-2] 모델 구성 및 학습하기**

- 데이터 하나에서 잘라서 씀


### 1. 필요한 패키지 가져오기

In [10]:
# 3개의 특징으로 예측하기 (3점슛, 블로킹, 리바운드) - kNN library 사용
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

### 2. 모델 객체 선언 및 학습하기


In [11]:
# 모델만들기 - knn으로 선언 : KNeighborsClassifier()를 분류기로 지정하기, 이때 이웃수는 5(n_neighbors=5)
model = KNeighborsClassifier()              #... 코드 작성
# 모델 학습하기 : X_train, y_train 을 사용하여 모델 학습하기
model.fit(X_train, y_train)                #... 코드 작성

KNeighborsClassifier()

### 3. 모델 예측하기

In [12]:
# 모델의 predict()를 사용하여 test 데이터로 예측하고 결과를 pred에 저장하기
pred = model.predict(X_test)              #... 코드 작성

pred                   # check ground_truth with knn prediction

array(['C', 'SG', 'C', 'SG', 'C', 'C', 'C', 'SG', 'SG', 'C', 'SG', 'SG',
       'SG', 'C', 'C', 'SG', 'SG', 'C', 'SG', 'C'], dtype=object)

In [13]:
comparison = pd.DataFrame({'prediction':pred, 'ground_truth':y_test.values})
comparison

,prediction,ground_truth
0,C,C
1,SG,SG
2,C,C
3,SG,SG
4,C,C
5,C,C
6,C,C
7,SG,SG
8,SG,SG
9,C,C


### 4. 모델 평가하기

In [14]:
# check accuracy - 실제값과 결과물을 비교하여 구함
#print("accuracy : " + str( accuracy_score(y_test.values.ravel(), pred)) )

print("accuracy : " + str( accuracy_score(y_test, pred)) )

accuracy : 0.95


## **[방법2-1] 데이터셋 준비하기**



### 1. 라이브러리 및 데이터 로드하기

In [15]:
import pandas as pd
import numpy as np

# import kNN library
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# sklearn 모델의  동일  결과  출력을  위해  선언합니다.
np.random.seed(5)

In [16]:
# 데이터 읽어와 살펴보기
df1 = pd.read_csv("basketball.csv")
df1.head()

,Player,Pos,3P,TRB,BLK
0,Denzel Valentine,SG,1.3,2.6,0.1
1,Kyle Korver,SG,2.4,2.8,0.3
2,Troy Daniels,SG,2.1,1.5,0.1
3,Tim Hardaway,SG,1.9,2.8,0.2
4,Dewayne Dedmon,C,0.0,6.5,0.8


### 2. 특성/타켓 항목 정하고 학습/테스트 데이터셋 준비하기

In [17]:
# 특징 데이터 준비하기
X = df1.drop(columns=['Player','Pos'])
X

,3P,TRB,BLK
0,1.3,2.6,0.1
1,2.4,2.8,0.3
2,2.1,1.5,0.1
3,1.9,2.8,0.2
4,0.0,6.5,0.8
...,...,...,...
95,3.3,2.7,0.5
96,1.6,2.8,0.2
97,0.0,5.7,0.7
98,1.9,3.2,0.3


In [18]:
# 타겟 데이터 준비하기
y = df1['Pos']             #... 코드 작성
y.head()

0    SG
1    SG
2    SG
3    SG
4     C
Name: Pos, dtype: object

In [19]:
# 학습, 테스트 데이터로 분리하기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [20]:
X_train.head()

,3P,TRB,BLK
55,1.9,3.1,0.1
88,1.4,3.0,0.1
26,0.3,10.4,1.0
42,1.6,2.1,0.1
69,1.4,2.4,0.1


In [21]:
y_train.head()

55    SG
88    SG
26     C
42    SG
69    SG
Name: Pos, dtype: object

### 3. 모델 객체 생성하고 학습하기

In [22]:
# 모델 객체 선언하기
model = KNeighborsClassifier(n_neighbors=5)

# 모델 학습하기
model.fit(X_train, y_train)

KNeighborsClassifier()

In [23]:
y_test.values

array(['SG', 'C', 'SG', 'SG', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'SG', 'C',
       'C', 'SG', 'SG', 'C', 'C', 'C', 'SG', 'SG'], dtype=object)

### 4. 모델 예측하기

In [24]:
y_pred = model.predict(X_test)
y_pred

array(['SG', 'C', 'SG', 'SG', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'SG', 'C',
       'C', 'SG', 'SG', 'C', 'C', 'C', 'SG', 'SG'], dtype=object)

In [25]:
# check ground_truth with knn prediction
comparison = pd.DataFrame({'prediction':y_pred, 'ground_truth':y_test.values.ravel()})
comparison

,prediction,ground_truth
0,SG,SG
1,C,C
2,SG,SG
3,SG,SG
4,C,C
5,SG,SG
6,C,C
7,C,C
8,SG,SG
9,SG,SG


### 5. 모델 평가(정확도 알아보기)

In [26]:
# check accuracy
print("accuracy : " + str(accuracy_score(y_test.values, y_pred)))

accuracy : 1.0


In [27]:
df_matrix = confusion_matrix(y_test, y_pred)
df_matrix

array([[ 9,  0],
       [ 0, 11]])

In [28]:
# Classification Report 출력하기
df_conf = pd.DataFrame(df_matrix).rename(index={0:'실제값(N)', 1:'실제값(P)'},
                               columns ={0:'예측값(N)', 1:'예측값(P)'})
df_conf

,예측값(N),예측값(P)
실제값(N),9,0
실제값(P),0,11


In [29]:
#*  입력된 데이터(y_test) 중 정확히 예측한 데이터(y_pred)의 정확도(accuracy), 정밀도 (precision), 재현율(recall), F1 점수(F1-score)를 확인한다.
print("정확도: ", accuracy_score(y_test, y_pred) )     # ... 코드 작성
print("정밀도: ", precision_score(y_test, y_pred, average='macro'))
print("재현율: ", recall_score(y_test, y_pred, average='macro'))     # ... 코드 작성
print("F1-Score: ", f1_score(y_test, y_pred, average='macro'))     # ... 코드 작성

정확도:  1.0
정밀도:  1.0
재현율:  1.0
F1-Score:  1.0


### 교차검증 방식 사용

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd


df = pd.read_csv("basketball.csv")

X = df[['3P', 'BLK', 'TRB']]  # 예측 변수
y = df['Pos']                 # 타겟 변수

model = KNeighborsClassifier(n_neighbors=7)

# 교차 검증 실행 (cv: 교차 검증의 분할 수)
cross_validation_scores = cross_val_score(model, X, y, cv=5)

print('교차 검증의 정확도:', cross_validation_scores)
print('교차 검증의 평균 정확도:', cross_validation_scores.mean())

교차 검증의 정확도: [0.9  0.9  0.85 0.9  0.95]
교차 검증의 평균 정확도: 0.9


In [31]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

df = pd.read_csv("basketball.csv")

# 특성과 레이블 분리
X = df[['3P', 'TRB', 'BLK']]
y = df['Pos']

# K-Neighbors Classifier 설정
model = KNeighborsClassifier(n_neighbors=3)

# K-Fold 교차 검증 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 점수를 저장할 리스트 초기화
precision_scores = []
recall_scores = []
f1_scores = []
accuracy_scores = []

# 교차 검증 실행
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # 모델 학습
    model.fit(X_train, y_train)

    # 예측
    y_pred = model.predict(X_test)

    # 성능 평가
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)

    # 점수 저장
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    accuracy_scores.append(accuracy)

# 평균 점수 출력
print("평균 정밀도: {:.2f}".format(np.mean(precision_scores)))
print("평균 재현율: {:.2f}".format(np.mean(recall_scores)))
print("평균 F1 점수: {:.2f}".format(np.mean(f1_scores)))
print("평균 정확도: {:.2f}".format(np.mean(accuracy_scores)))

평균 정밀도: 0.90
평균 재현율: 0.89
평균 F1 점수: 0.89
평균 정확도: 0.90
